# 0. Contents

1. Departments average number of orders

2. Creation or max order columns

3. Creation of customer loyalty flag

4. Customer habit by loyalty flag analysis

5. Creation od mean_price columns

6. Creation os spending flag

7. Creation of med_order_freq column

8. Creation of customer frequency flag

# 1. Importing Libraries and Data

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
# creating main path
path = r'C:\Users\muffi\OneDrive\CareerFoundry\Instacart Basket Analysis 2023'

In [3]:
# importing merged orders and products dataset
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v2.pkl'))

# 2. Departments' Average Order Number

In [4]:
# calculating the average number of orders for each department
ords_prods_merged.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

The averages of the population differ slight from those of the sample. In most cases the population has more orders on average than the sample suggested

# 3. Customer Loyalty Flag

In [5]:
# creating a col of the total number of orders for a user
# group by user
# calculate the max order number with the tranform function
# name the col max_order 
ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
# checking the new col
ords_prods_merged[['user_id','max_order']].head()

,user_id,max_order
0,138,32
1,138,32
2,709,5
3,764,3
4,764,3


In [7]:
# loyalty flag for "Loyal customer"
# max_order > 40 
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'

In [8]:
# loyalty flag for "regular customer"
# 10 < max_order <= 40 
ords_prods_merged.loc[(ords_prods_merged['max_order'] > 10) & (ords_prods_merged['max_order'] <= 40), 'loyalty_flag'] = 'Regular Customer'

In [9]:
# loyalty flag for "new customer"
# max_order >= 10 
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [10]:
# checking the loyalty flag
ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular Customer    15876776
Loyal Customer      10284093
New Customer         6243990
Name: loyalty_flag, dtype: int64

In [11]:
# calculating basic statistics of each loyalty group
# group by loyalty flag 
ords_prods_merged.groupby('loyalty_flag')

# 4. Customer Habits

In [12]:
# comparing spending habits of different customer types
# group by loyalty_flag

# average product price for each customer group
ords_prods_merged.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal Customer      10.386336
New Customer        13.294670
Regular Customer    12.495717
Name: prices, dtype: float64

In [13]:
# basic statistice for the product price of each customer group (mean should match mean calculated above)
ords_prods_merged.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal Customer,10284093.0,10.386336,328.017787,1.0,4.2,7.4,11.2,99999.0
New Customer,6243990.0,13.294670,597.560299,1.0,4.2,7.4,11.3,99999.0
Regular Customer,15876776.0,12.495717,539.720919,1.0,4.2,7.4,11.3,99999.0


The min, max, and percentiles are the samefor all three customer grouping (I wonder if this this the case because of the outrageously high max.

New customers have the highest average product price following by regular customers then loyal customers. This could mean that new customer are more likely to buy more expensive things. It could also indicate that the more customers use the app, the more compforable they get with making ordinaly, inexpensive purchases rather than important big ticket items. 

# 5. Marketing Spending Flag

In [14]:
# user flag for average product price across all their purchases
# group by user
# looking at average product price 
# mean_price < 10 is "Low Spender"
# mean_price >= 10 is "High Spender"

In [15]:
# creating a col of the users' average product price 
ords_prods_merged['mean_price'] = ords_prods_merged.groupby('user_id')['prices'].transform(np.mean)

In [16]:
# cheaking the mean_price col 
ords_prods_merged[['user_id','mean_price', 'max_order']].head()

,user_id,mean_price,max_order
0,138,6.935811,32
1,138,6.935811,32
2,709,7.930208,5
3,764,4.972414,3
4,764,4.972414,3


In [17]:
# creating Low Spender flag (mean_price < 10)
ords_prods_merged.loc[ords_prods_merged['mean_price'] < 10, 'spending_flag'] = "Low Spender"

In [18]:
# creating High Spender flag (mean_price >= 10)
ords_prods_merged.loc[ords_prods_merged['mean_price'] >= 10, 'spending_flag'] = "High Spender"

In [19]:
ords_prods_merged['spending_flag'].value_counts(dropna = False)

Low Spender     31770614
High Spender      634245
Name: spending_flag, dtype: int64

# 6. Customer Frequency Flag

In [20]:
# ordering frequency tag 
# group by user_id
# looking at days_since_prior_order
# med_order_freq > 20 is "Non-freqeunt Customer"
# 10 < med_order_freq <= 20 is "Regular Customer"
# med_order_freq <= 10 is "Freqeunt Customer"

In [21]:
# creating a col of a users' median order frequency 
ords_prods_merged['med_order_freq'] = ords_prods_merged.groupby('user_id')['days_since_prior_order'].transform(np.median)

In [22]:
# cheacking the med_order_freq col
ords_prods_merged[['user_id', 'med_order_freq']].head()

,user_id,med_order_freq
0,138,8.0
1,138,8.0
2,709,6.0
3,764,9.0
4,764,9.0


In [23]:
# creating freqeuncy flag for non-frequent customers
ords_prods_merged.loc[ords_prods_merged['med_order_freq'] > 20, 'freq_flag'] = 'Non-freqeunt Customer'

In [24]:
# creating freqeuncy flag for regular customers
ords_prods_merged.loc[(ords_prods_merged['med_order_freq'] > 10) & (ords_prods_merged['med_order_freq'] <= 20), 'spending_flag'] = 'Regular Customer'

In [26]:
# creating freqeuncy flag for frequent customers
ords_prods_merged.loc[ords_prods_merged['med_order_freq'] <= 10, 'freq_flag'] = 'Freqeunt Customer'

In [27]:
# checking the spending flag 
ords_prods_merged['freq_flag'].value_counts(dropna = False)

Freqeunt Customer        22796659
NaN                       6921472
Non-freqeunt Customer     2686728
Name: freq_flag, dtype: int64

In [28]:
# marking the strange prices as missing
ords_prods_merged.loc[ords_prods_merged['prices'] > 100, 'prices'] = np.nan

# 7. Exporting Data

In [29]:
ords_prods_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v3.pkl'))